In [ ]:
%run -i ../python/common.py
publish=False

if not publish:
    # cleanup any old state
    bashCmds('''[[ -d lec3 ]] && rm -rf mydir
    [[ -a myinfo ]] && rm myinfo''')
else:
    bashCmds('''rm -rf ~/*''')
    
closeAllOpenTtySessions()


In [ ]:
appdir=os.getenv('HOME')
appdir=appdir + "/parser"
TermShellCmd("ls ")
output = runTermCmd("[[ -d " + appdir + " ]] &&  rm -rf "+ appdir + 
             ";cp -r ../src/parser " + appdir)
bash = BashSession(cwd=appdir)

(cont:gs:tools:testing)=
# Testing

Most students try to solve the complicated programs for this course by implementing them, and then manually testing their program.  This is, possible...., but incredibly challenging.  Proper unit and integration tests will, on the other hand, save you an enormous number of hours in this course. 

## Unit Tests

As you develop your program, you are going to write many individual functions.  For each, think about how to write simple tests to see if the function does what you expect it to.  For example, one of the assignments we often hand out is a user level thread scheduler.  To get it to work, you need to set up the stack of the thread properly so that when the thread completes it calls a routine to exit. If you write a simple test at the beginning to test what happens after a thread completes, it will take a few minutes to identify and fix any bugs.  Many students, instead, spend hours trying to identify the problem manually when they have a running scheduler, with timer interrupts causing threads to switch between themselves.  

Lets see some simple examples, we have already shown a script that runs a set of test programs ({numref}`run_tests`), and a makefile ({numref}`make_parser`)to invoke that script on a parser.  The first step on developing a parser is to define its interface, and a set of tests against that interface.  To do this, we define the data structures and functions that the parser will expose to its clients in a header file shown in {numref}`myshell_parser_h`.  

% The block below creates a nice labeled Listing in the html, but not in the jupyter notebook view.
```{literalinclude} /src/parser/myshell_parser.h
:linenos:
:language: make
:caption: Header file for parser for shell
:name: myshell_parser_h
```

In [ ]:
# This cell is removed in the html, but displays the code listing in the Jupyter notebook. 
file = appdir + "/myshell_parser.h"
text_file = open(file, "r")
data = text_file.read()
data = numberLines(data)
text_file.close()                                                                                             
md_text = '''                                                                                                                        
``` ''' + "c" + '''                                                                                                                     
''' + data + '''                                                                                                                         
```                                                                                                                                      
'''
display (Markdown(md_text))
#display (Markdown('<font size=".5rem">' + md_text +'</font>'))

In this header file, we define everything that test programs, and eventually the shell will need to use to call the parser we will develop.  The routines `pipeline_build` returns a `pipeline` struct with a flag that indicates if the whole pipleline is in the background, and points to a linked list of `pipeline_commands`.   Each `pipeline_command` contains a boolean indicates if the command is in the background, and an array where the first element is the command and the other elements are arguments to that command. Our first implementation of `myshell_parser.c` (see {numref}`myshell_parser_c`) simply returns error messages.  

% The block below creates a nice labeled Listing in the html, but not in the jupyter notebook view.
```{literalinclude} /src/parser/myshell_parser.c
:linenos:
:language: make
:caption: Initial implementation of 
:name: myshell_parser_c
```

In [ ]:
# This cell is removed in the html, but displays the code listing in the Jupyter notebook. 
file = appdir + "/myshell_parser.c"
text_file = open(file, "r")
data = text_file.read()
data = numberLines(data)
text_file.close()                                                                                             
md_text = '''                                                                                                                        
``` ''' + "c" + '''                                                                                                                     
''' + data + '''                                                                                                                         
```                                                                                                                                      
'''
display (Markdown(md_text))
#display (Markdown('<font size=".5rem">' + md_text +'</font>'))

Before implementing any functionality we write tests on what we expect a correct implementation to do.  For example, {numref}`test_simple_input_c` shows a test that calls the parser `pipeline_build` with a single command `ls`, asserts what it expects the state of a correct execution of the parser is. 

% The block below creates a nice labeled Listing in the html, but not in the jupyter notebook view.
```{literalinclude} /src/parser/test_simple_input.c
:linenos:
:language: c
:caption: Test of a simple 
:name: test_simple_input_c
```

In [ ]:
# This cell is removed in the html, but displays the code listing in the Jupyter notebook. 
file = appdir + "/test_simple_input.c"
text_file = open(file, "r")
data = text_file.read()
data = numberLines(data)
text_file.close()                                                                                             
md_text = '''                                                                                                                        
``` ''' + "c" + '''                                                                                                                     
''' + data + '''                                                                                                                         
```                                                                                                                                      
'''
display (Markdown(md_text))
#display (Markdown('<font size=".5rem">' + md_text +'</font>'))

As another example, {numref}`test_pipe_input_c` runs a simple example of two commands with a pipe between them.  

% The block below creates a nice labeled Listing in the html, but not in the jupyter notebook view.
```{literalinclude} /src/parser/test_simple_pipe.c
:linenos:
:language: c
:caption: Test of a simple pipe
:name: test_simple_pipe_c
```

In [ ]:
# This cell is removed in the html, but displays the code listing in the Jupyter notebook. 
file = appdir + "/test_simple_pipe.c"
text_file = open(file, "r")
data = text_file.read()
data = numberLines(data)
text_file.close()                                                                                             
md_text = '''                                                                                                                        
``` ''' + "c" + '''                                                                                                                     
''' + data + '''                                                                                                                         
```                                                                                                                                      
'''
display (Markdown(md_text))
#display (Markdown('<font size=".5rem">' + md_text +'</font>'))

We have already provided examples of how you can invoke the unit test programs from a shell scripts ({numref}`run_tests`), and shown how that can in turn by automatically invoked by make ({numref}`make_parser`) so that every time you make a change the makefile will automatically re-run all the tests.  Now, given the above test files, if we type make with that makefile, the `all` rule will run the `check` rule which will recompile any required software and then invoke the shell script to run all the tests. In our case, both test programs assert because the pipeline returned is `NULL`.

In [ ]:
bash.run("make clean ; make")

Unit tests are specific to the particular interfaces of the functions you write.  We expect students to write their own unit tests, since the internal functions of their application are up to them to design.  For the parser, we have defined the functions that we want you to implement, so that we can give you some example of unit tests.  However, we would recommend that a good parser implementation should start by implementing a lexer, that converts each syntactical element of the input into a set of tokens.  If you do that, you should add the new interface to the `.h` file and new tests to make sure that your lexing functionality works.

You should never delete your tests.  For example, the parser will eventually become the first shell assignment for many courses based on this book.  You will keep adding new functionality, and as you do, you will find that you will find bugs in your parser.  If your makefile doesn't keep running all the old tests, you are very likely to be introducing bugs... 

## Integration Tests

Integration tests use the public interfaces end-to-end.  For example, the tests we run with gradescope to automatically test if your programs are correct are all integration tests.  We hope you will share your integration tests with the class, and in the BU version of the course, we will often add tests provided by students to the test suite we use in gradescope; the best way you can have tests you know you will pass is to contribute tests to us.  In many cases, you can write integration tests in the same we described above, but calling the public interfaces of libraries.  If the task is to develop a program, you can use, for example, shell scripts, with the output redirected to a file, to run tests, compare the result to an expect result, and raise an error if the results do not agree with the expected ones. 
